<a href="https://colab.research.google.com/github/HillaAnku/HillaAnku/blob/master/cont_vars_exploration_credit_risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Continous Variables Processing And Selection**

This notebook takes a walk through the processing, analysis and selection of continous variables for the credit risk model of the lending club dataset. There are about 2.2M+ records in this dataset.

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler

In [0]:
# load dataset

lendingclub_df = pd.read_csv('/content/drive/My Drive/python_for_data_scientists/lendingclub.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# display all the columns of the dataset

pd.options.display.max_columns = None

I try to separate continous and categorical variables into two different dataframes which will be analyzed separately. The variables selected subsequently will be used in building the credit model. 

In [0]:
# Make a list of continius variables

cont_vars = []
for column_name in lendingclub_df.columns:
    if lendingclub_df[column_name].dtype != 'object':
        cont_vars.append(column_name)

In [0]:
# Make a list of categorical variables

cat_vars = []
for column_name in lendingclub_df.columns:
    if lendingclub_df[column_name].dtype == 'object':
        cat_vars.append(column_name)

In [0]:
cat_vars

['term',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'initial_list_status',
 'last_pymnt_d',
 'next_pymnt_d',
 'last_credit_pull_d',
 'application_type',
 'verification_status_joint',
 'sec_app_earliest_cr_line',
 'hardship_flag',
 'hardship_type',
 'hardship_reason',
 'hardship_status',
 'hardship_start_date',
 'hardship_end_date',
 'payment_plan_start_date',
 'hardship_loan_status',
 'disbursement_method',
 'debt_settlement_flag',
 'debt_settlement_flag_date',
 'settlement_status',
 'settlement_date']

In [0]:
# Create a dataframe of continous variables only

cont_vars_df = lendingclub_df[cont_vars]

In [0]:
# Create a dataframe of categorical variables only

cat_vars_df = lendingclub_df[cat_vars]

Loan status is the target variable for the credit risk model. It appears in the dataset as a categorical variable with 9 distinct string descritions that describe the status of a loan. Loan status is added to continous variables to be processed and analyzed in terms of its relationship with the continous variables that will later be used as loan status predictors in the risk model. 

In [0]:
# Add in the target variable loan status

cont_vars_df = pd.concat([cont_vars_df, lendingclub_df['loan_status']], axis = 1)

emp-length, issue-d, earliest-cr-line and term are supposed to be continous variables in the data. But they were recorded in the data as categorical variables. They are also added into the continous variables dataframe for processing and analysis.

In [0]:
# Add emp_length to contnous variables

cont_vars_df = pd.concat([cont_vars_df, lendingclub_df['emp_length']], axis = 1)

In [0]:
# Add issue_d to continous variables

cont_vars_df = pd.concat([cont_vars_df, lendingclub_df['issue_d']], axis = 1)

In [0]:
# Add earliest_cr_line to continous variables

cont_vars_df = pd.concat([cont_vars_df, lendingclub_df['earliest_cr_line']], axis = 1)

In [0]:
# Add term to continous variables

cont_vars_df = pd.concat([cont_vars_df, lendingclub_df['term']], axis = 1)

In [0]:
cont_vars_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,url,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term,loan_status,emp_length,issue_d,earliest_cr_line,term
0,NaN,NaN,2500,2500,2500.0,13.56,84.92,55000.0,NaN,18.24,0.0,1.0,NaN,45.0,9.0,1.0,4341,10.3,34.0,2386.02,2386.02,167.02,167.02,113.98,53.04,0.0,0.0,0.0,84.92,0.0,NaN,1,NaN,NaN,0.0,0.0,16901.0,2.0,2.0,1.0,2.0,2.0,12560.0,69.0,2.0,7.0,2137.0,28.0,42000.0,1.0,11.0,2.0,9.0,1878.0,34360.0,5.9,0.0,0.0,140.0,212.0,1.0,1.0,0.0,1.0,NaN,2.0,NaN,0.0,2.0,5.0,3.0,3.0,16.0,7.0,18.0,5.0,9.0,0.0,0.0,0.0,3.0,100.0,0.0,1.0,0.0,60124.0,16901.0,36500.0,18124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Current,10+ years,Dec-2018,Apr-2001,36 months
1,NaN,NaN,30000,30000,30000.0,18.94,777.23,90000.0,NaN,26.52,0.0,0.0,71.0,75.0,13.0,1.0,12315,24.2,44.0,29387.75,29387.75,1507.11,1507.11,612.25,894.86,0.0,0.0,0.0,777.23,0.0,NaN,1,NaN,NaN,0.0,1208.0,321915.0,4.0,4.0,2.0,3.0,3.0,87153.0,88.0,4.0,5.0,998.0,57.0,50800.0,2.0,15.0,2.0,10.0,24763.0,13761.0,8.3,0.0,0.0,163.0,378.0,4.0,3.0,3.0,4.0,NaN,4.0,NaN,0.0,2.0,4.0,4.0,9.0,27.0,8.0,14.0,4.0,13.0,0.0,0.0,0.0,6.0,95.0,0.0,1.0,0.0,372872.0,99468.0,15000.0,94072.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Current,10+ years,Dec-2018,Jun-1987,60 months
2,NaN,NaN,5000,5000,5000.0,17.97,180.69,59280.0,NaN,10.51,0.0,0.0,NaN,NaN,8.0,0.0,4599,19.1,13.0,4787.21,4787.21,353.89,353.89,212.79,141.10,0.0,0.0,0.0,180.69,0.0,NaN,1,NaN,NaN,0.0,0.0,110299.0,0.0,1.0,0.0,2.0,14.0,7150.0,72.0,0.0,2.0,0.0,35.0,24100.0,1.0,5.0,0.0,4.0,18383.0,13800.0,0.0,0.0,0.0,87.0,92.0,15.0,14.0,2.0,77.0,NaN,14.0,NaN,0.0,0.0,3.0,3.0,3.0,4.0,6.0,7.0,3.0,8.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,136927.0,11749.0,13800.0,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Current,6 years,Dec-2018,Apr-2011,36 months
3,NaN,NaN,4000,4000,4000.0,18.94,146.51,92000.0,NaN,16.74,0.0,0.0,NaN,NaN,10.0,0.0,5468,78.1,13.0,3831.93,3831.93,286.71,286.71,168.07,118.64,0.0,0.0,0.0,146.51,0.0,NaN,1,NaN,NaN,0.0,686.0,305049.0,1.0,5.0,3.0,5.0,5.0,30683.0,68.0,0.0,0.0,3761.0,70.0,7000.0,2.0,4.0,3.0,5.0,30505.0,1239.0,75.2,0.0,0.0,62.0,154.0,64.0,5.0,3.0,64.0,NaN,5.0,NaN,0.0,1.0,2.0,1.0,2.0,7.0,2.0,3.0,2.0,10.0,0.0,0.0,0.0,3.0,100.0,100.0,0.0,0.0,385183.0,36151.0,5000.0,44984.0,N

To avoid kernel crushing issues, I take a sample of 200000 of the 2M+ instances for processing and subsequent analysis. All records will be used in the modeling stage.

In [0]:
# Take a sample of the data

cont_sample = cont_vars_df.sample(200000)

In [0]:
cont_sample.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 2181401 to 1818638
Data columns (total 114 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   id                                          0 non-null       float64
 1   member_id                                   0 non-null       float64
 2   loan_amnt                                   200000 non-null  int64  
 3   funded_amnt                                 200000 non-null  int64  
 4   funded_amnt_inv                             200000 non-null  float64
 5   int_rate                                    200000 non-null  float64
 6   installment                                 200000 non-null  float64
 7   annual_inc                                  200000 non-null  float64
 8   url                                         0 non-null       float64
 9   dti                                         199879 non-null  f

In the following lines, I process all the categorical variables that are supposed to be continous variables by cleaning them up, removing unwanted characters, renaming and changing their dtypes accordingly.

In [0]:
# Strip unwanted string characers from emp_length column

cont_sample['emp_length_int'] = cont_sample['emp_length'].str.replace(' years', '')
cont_sample['emp_length_int'] = cont_sample['emp_length_int'].str.replace('< 1 year', str(0))
cont_sample['emp_length_int'] = cont_sample['emp_length_int'].str.replace('1 year', str(1))
cont_sample['emp_length_int'] = cont_sample['emp_length_int'].str.replace('10\+', '10')

In [0]:
# convert emp_length_int(categorical) to numerical data

cont_sample['emp_length_int'] = pd.to_numeric(cont_sample['emp_length_int'])

In [0]:
# Strip unwanted string characters from the 'term' variable

cont_sample['term_int'] = cont_sample['term'].str.replace(' ', '')
cont_sample['term_int'] = cont_sample['term_int'].str.replace('months', '')

In [0]:
# convert the categorical 'term_int' variable to a numerical one

cont_sample['term_int'] = pd.to_numeric(cont_sample['term_int'])

In [0]:
# Do the same for the issue_d (date loan was issued), convert the column to a date type

cont_sample['_since_issue_d'] = pd.to_datetime(cont_sample['issue_d'])

In [0]:
# Issue_d is a date type, convert to date type

cont_sample['mths_since_loan_issue'] = round(pd.to_numeric((pd.to_datetime('2020-04-18') - cont_sample['_since_issue_d'])/np.timedelta64(1, 'M')))

In [0]:
# earliest_cr_line is stored as a string but it should be a date type. Convert it to date type

cont_sample['tm_of_earliestcr'] = pd.to_datetime(cont_sample['earliest_cr_line'])

In [0]:
# Instead of showing the date time in days, convert to number of months

cont_sample['mths_since_earliest_cr'] = round(pd.to_numeric((pd.to_datetime('2019-03-18') - cont_sample['tm_of_earliestcr'])/np.timedelta64(1, 'M')))

In [0]:
# Drop the original categorical vriables and keep their numerical versions only

cont_sample = cont_sample.drop(['emp_length', 'issue_d', 'earliest_cr_line', 'term', '_since_issue_d', 'tm_of_earliestcr'], axis = 1)

At this point, all variables are numerical except loan status which will be taken care of later in the analysis. I, therefore, proceed to analyze the variables for relatonships with the target variable.

In [0]:
pd.set_option('display.max_rows', None)

In [0]:
# check for missing values

cont_sample.isnull().sum()*100/cont_sample.isnull().count()

id                                            100.0000
member_id                                     100.0000
loan_amnt                                       0.0000
funded_amnt                                     0.0000
funded_amnt_inv                                 0.0000
int_rate                                        0.0000
installment                                     0.0000
annual_inc                                      0.0000
url                                           100.0000
dti                                             0.0605
delinq_2yrs                                     0.0020
inq_last_6mths                                  0.0020
mths_since_last_delinq                         51.2835
mths_since_last_record                         84.1155
open_acc                                        0.0020
pub_rec                                         0.0020
revol_bal                                       0.0000
revol_util                                      0.0790
total_acc 

There are a lot of missing values in the data. Because we have a lot of data, it is okay to drop variables that have 60% or more missing values. I proceed to make a list of such variables and drop them from the analysis.

In [0]:
# Make a list of variables with 60% or more missing values

drop_missing_values = []
for column_name in cont_sample.columns:
    numerator = cont_sample[column_name].isnull().sum()
    denominator = numerator + cont_sample[column_name].count()
    if (numerator*100/denominator) >= 60:
        drop_missing_values.append(column_name)

In [0]:
# Drop columns with 60% or more of missing values

cont_sample.drop(drop_missing_values, axis = 1, inplace = True)

In [0]:
# Quick view dataframe

cont_sample.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,loan_status,emp_length_int,term_int,mths_since_loan_issue,mths_since_earliest_cr
2181401,12500,12500,12500.0,9.44,400.07,61000.0,24.89,0.0,0.0,NaN,7.0,0.0,8216,46.9,20.0,0.00,0.00,13328.079552,13328.08,12500.00,828.08,0.0,0.00,0.0000,10140.63,0.0,1,0.0,0.0,54555.0,0.0,3.0,2.0,3.0,8.0,46339.0,79.0,1.0,2.0,5833.0,61.0,17500.0,1.0,2.0,2.0,5.0,7794.0,1284.0,86.5,0.0,0.0,154.0,111.0,9.0,8.0,1.0,35.0,8.0,0.0,2.0,2.0,2.0,3.0,14.0,4.0,5.0,2.0,7.0,0.0,0.0,0.0,3.0,100.0,50.0,0.0,0.0,71229.0,54555.0,9500.0,53729.0,Fully Paid,5.0,36,29.0,170.0
317128,19000,19000,19000.0,7.46,590.67,95000.0,34.77,0.0,0.0,NaN,23.0,0.0,63285,60.0,49.0,14639.71,14639.71,5300.280000,5300.28,4360.29,939.99,0.0,0.00,0.0000,590.67,0.0,1,0.0,0.0,107624.0,0.0,12.0,0.0,1.0,17.0,44339.0,64.0,1.0,1.0,17740.0,62.0,100900.0,1.0,0.0,0.0,2.0,4679.0,30415.0,65.1,0.0,0.0,282.0,293.0,9.0,9.0,0.0,9.0,17.0,0.0,8.0,8.0,8.0,10.0,34.0,11.0,15.0,8.0,23.0,0.0,0.0,0.0,1.0,100.0,62.5,0.0,0.0,169728.0,107624.0,93700.0,68828.0,Current,5.0,36,24.0,304.0
1310174,16000,16000,16000.0,7.49,497.63,120000.0,8.11,0.0,0.0,NaN,12.0,1.0,14175,65.9,27.0,0.00,0.00,16356.300000,16356.30,16000.00,356.30,0.0,0.00,0.0000,15367.70,0.0,1,0.0,0.0,244929.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21500.0,NaN,NaN,NaN,5.0,22266.0,3794.0,75.0,0.0,0.0,101.0,226.0,7.0,7.0,1.0,25.0,7.0,0.0,5.0,7.0,6.0,16.0,3.0,10.0,23.0,7.0,12.0,0.0,0.0,0.0,1.0,100.0,60.0,1.0,0.0,260340.0,27063.0,15200.0,15462.0,Fully Paid,10.0,36,63.0,277.0
605130,10200,10200,10200.0,7.39,316.77,93000.0,12.27,0.0,0.0,75.0,6.0,0.0,3098,15.6,20.0,0.00,0.00,5016.970000,5016.97,3426.20,683.43,0.0,907.34,163.3212,316.77,0.0,1,0.0,0.0,154462.0,0.0,2.0,2.0,2.0,9.0,25343.0,78.0,0.0,0.0,1979.0,55.0,19800.0,1.0,3.0,2.0,4.0,25744.0,16702.0,15.6,0.0,0.0,129.0,71.0,29.0,9.0,4.0,29.0,11.0,0.0,2.0,2.0,3.0,4.0,9.0,3.0,7.0,2.0,6.0,0.0,0.0,0.0,4.0,95.0,0.0,0.0,0.0,180008.0,28441.0,19800.0,32433.0,Charged Off,10.0,36,47.0,163.0
421895,24000,24000,24000.0,11.98,533.63,72000.0,12.92,0.0,0.0,NaN,7.0,0.0,7002,46.7,11.0,20600.47,20600.47,5853.960000,5853.96,3399.53,2454.43,0.0,0.00,0.0000,533.63,0.0,1,0.0,0.0,39143.0,1.0,3.0,1.0,1.0,6.0,32141.0,83.0,0.0,1.0,4558.0,73.0,15000.0,0.0,0.0,1.0,2.0,5592.0,1742.0,72.3,0.0,0.0,122.0,186.0,23.0,6.0,0.0,39.0,6.0,0.0,1.0,3.0,1.0,1.0,5.0,4.0,6.0,3.0,7.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,53811.0,39143.0,6300.0,38811.0,Current,3.0,60,26.0,200.0


For the remaining variables, I proceed to fill missing values with the mean value of that column. Doing so will make sure that any effects are smoothened and distribueted evenly across.

In [0]:
# Fill missing values in each column with the mean of that column

for column_name in cont_sample.columns:
  if cont_sample[column_name].dtype != 'object':
    cont_sample[column_name].fillna(cont_sample[column_name].mean(), inplace = True)

I now turn to analyzing the target variable in this case loan status. It is the only categorical variable in the data at this point. I begin analysis by looking a some summary stats. 

The loan status variable has 9 distinct loan statuses. Some of these statuses depict loan default and others do not. I binary encode this column and rename it as 'good_bad'. If a loan status has any of the following statuses: 'Charged Off, Default, Late (16-30 days), Does not meet the credit policy. Status: Charged Off' is encoded as 0 which depicts a bad loan or simply that the customer defaulted. All others statuses are encoded as 1 meaning a good loan or customer has not defaulted.

In [0]:
# Binary encode the loan status target variable 

cont_sample['good_bad'] = np.where(cont_sample['loan_status'].isin(['Charged Off', 'Default', 
                                                                   'Late (16-30 days)',
                                                                   'Does not meet the credit policy. Status:Charged Off',
                                                                   ]), 0, 1)

In [0]:
# Get a summary count of good and bad loans in each column 

cont_sample.groupby('good_bad').count()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,loan_status,emp_length_int,term_int,mths_since_loan_issue,mths_since_earliest_cr
good_bad,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719,23719
1,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281,176281


From the summary, it can be seen that there are more good loans than there are bad loans. This is a welcome news for lenders as more of their loans are getting paid back than customers defaulting on their loans. However, a good number of customers still default on their loans. About 13% of loans are in default. This is certainly a number that lending club or any lending institution would love to see go down some more. 

In [0]:
# loan status is binary encoded, it can now be droped from the analysis at this stage

cont_sample.drop('loan_status', axis = 1, inplace = True)

There is so much variation in the data because each variable is on a different scale. This may cause some issues for the normality of distributions. It may also overemphasize the impact of outliers, I scale the data to normalize and standardize it.

In [0]:
# Scale the data
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame

scaler = StandardScaler()
scaled_data = scaler.fit_transform(cont_sample)
cont_sample_scaled = pd.DataFrame(scaled_data, columns = cont_sample.columns)

In [0]:
# Quick view scaled data

cont_sample_scaled.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,emp_length_int,term_int,mths_since_loan_issue,mths_since_earliest_cr,good_bad
0,-0.275501,-0.275002,-0.272950,-0.753933,-0.169730,-0.112557,0.452850,-0.354608,-0.647547,0.000000,-0.815675,-0.349276,-0.371348,-0.138904,-0.347731,-0.588249,-0.588190,0.154094,0.155923,0.387343,-0.585498,-0.124503,-0.186906,-0.179068,1.137712,-0.120698,0.0,-0.061429,-0.133941,-0.550992,-1.041750e+00,9.374643e-02,1.816751,1.153398,-6.542568e-01,0.301935,5.703958e-01,-0.243686,-3.667681e-01,0.003858,2.400560e-01,-0.493669,-1.600691e-02,0.244125,-0.022814,0.153886,-0.353365,-0.620189,1.017159,-0.078864,-0.01813,0.544789,-0.739462,-0.292568,-0.032013,-0.297903,0.317378,0.171989,-0.377617,-0.732683,-1.089031,-0.926062,-1.020911,0.761983,-0.919355,-1.137479,-1.102347,-0.827926,-0.025772,-0.05209,-0.169742,0.515969,0.660280,0.213412,-0.350283,-0.126898,-0.598969,0.070209,-0.603847,0.222224,-0.258713,-0.635836,-0.878214,-0.654022,0.366814
1,0.430918,0.431597,0.433360,-1.162576,0.543166,0.108962,1.184307,-0.354608,-0.647547,0.000000,2.021875,-0.349276,2.047801,0.390809,2.067088,1.346325,1.346608,-0.657680,-0.656199,-0.592392,-0.543406,-0.124503,-0.186906,-0.179068,-0.464107,-0.120698,0.0,-0.061429,-0.133941,-0.219812,-1.041750e+00,3.899610e+00,-0.931437,-0.457888,-2.072949e-01,0.245712,-2.985498e-01,-0.243686,-8.611072e-01,2.681972,3.007098e-01,1.917544,-1.600691e-02,-0.699995,-1.089647,-0.808398,-0.542491,1.164568,0.257171,-0.078864,-0.01813,3.011209,1.162338,-0.292568,0.078140,-0.828083,-0.497529,1.784370,-0.377617,1.891165,0.714811,1.079778,0.492709,3.499749,0.601643,0.127234,0.750374,2.043717,-0.025772,-0.05209,-0.169742,-0.598512,0.660280,0.564188,-0.350283,-0.126898,-0.048899,1.132068,3.106599,0.557096,-0.258713,-0.635836,-1.110473,0.766171,0.366814
2,0.104878,0.105474,0.107371,-1.156384,0.195171,0.271843,-0.789441,-0.354608,-0.647547,0.000000,0.071059,1.414130,-0.109572,0.629383,0.235156,-0.588249,-0.588190,0.460309,0.462269,0.808620,-0.762947,-0.124503,-0.186906,-0.179068,2.014451,-0.120698,0.0,-0.061429,-0.133941,0.637047,1.241680e-16,-1.877937e-16,0.000000,0.000000,-1.764364e-16,0.000000,8.232306e-16,0.000000,2.195307e-16,0.000000,8.619423e-16,-0.378023,1.888674e-16,0.000000,0.000000,0.153886,0.525297,-0.466409,0.608754,-0.078864,-0.01813,-0.476463,0.462225,-0.409116,-0.142166,-0.297903,0.003952,-0.007165,-0.377617,0.579241,0.414171,0.411165,1.790097,-0.743788,0.384358,1.139005,0.441588,0.069463,-0.025772,-0.05209,-0.169742,-0.598512,0.660280,0.494033,2.403678,-0.126898,0.457126,-0.479879,-0.352665,-0.626479,1.129438,-0.635836,0.701146,0.480013,0.366814
3,-0.525465,-0.525030,-0.522875,-1.177023,-0.481295,0.095931,-0.481460,-0.354608,-0.647547,2.642155,-0.993022,-0.349276,-0.596178,-1.404556,-0.347731,-0.588249,-0.588190,-0.686328,-0.684860,-0.704823,-0.639905,-0.124503,1.059660,1.116638,-0.510049,-0.120698,0.0,-0

In [0]:
# Check to see if normalization had any effect

from scipy.stats import jarque_bera
from scipy.stats import normaltest

for column_name in cont_sample_scaled.columns:
    if cont_sample_scaled[column_name].dtype != 'object':
        jb_stats = jarque_bera(cont_sample_scaled[column_name])
        norm_stats = normaltest(cont_sample_scaled[column_name])
        print('\n' + column_name)
        print("Jarque-Bera test statistics is {0} and p value is {1}".format(jb_stats[0], jb_stats[1]))
        print("Normality test statistics is {0} and p value is {1}".format(norm_stats[0], norm_stats[1]))


loan_amnt
Jarque-Bera test statistics is 20501.357497681787 and p value is 0.0
Normality test statistics is 16229.41911369734 and p value is 0.0

funded_amnt
Jarque-Bera test statistics is 20552.19374148605 and p value is 0.0
Normality test statistics is 16259.156084231747 and p value is 0.0

funded_amnt_inv
Jarque-Bera test statistics is 20533.865268306206 and p value is 0.0
Normality test statistics is 16246.842672906225 and p value is 0.0

int_rate
Jarque-Bera test statistics is 22841.96522978704 and p value is 0.0
Normality test statistics is 17566.977500731846 and p value is 0.0

installment
Jarque-Bera test statistics is 37857.325182123124 and p value is 0.0
Normality test statistics is 26183.15296073437 and p value is 0.0

annual_inc
Jarque-Bera test statistics is 128565666017637.95 and p value is 0.0
Normality test statistics is 1110738.7623134048 and p value is 0.0

dti
Jarque-Bera test statistics is 21386108829.64644 and p value is 0.0
Normality test statistics is 496299.789

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1688: RuntimeWarning: invalid value encountered in double_scalars
  skewness = (1 / n * np.sum(diffx**3)) / (1 / n * np.sum(diffx**2))**(3 / 2.)
/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1689: RuntimeWarning: invalid value encountered in double_scalars
  kurtosis = (1 / n * np.sum(diffx**4)) / (1 / n * np.sum(diffx**2))**2



open_act_il
Jarque-Bera test statistics is 5202188.442310334 and p value is 0.0
Normality test statistics is 163200.81026122923 and p value is 0.0

open_il_12m
Jarque-Bera test statistics is 773381.8492503323 and p value is 0.0
Normality test statistics is 97153.8892213809 and p value is 0.0

open_il_24m
Jarque-Bera test statistics is 1051541.9069249115 and p value is 0.0
Normality test statistics is 99644.74476997521 and p value is 0.0

mths_since_rcnt_il
Jarque-Bera test statistics is 9041327.608426897 and p value is 0.0
Normality test statistics is 185610.2151169857 and p value is 0.0

total_bal_il
Jarque-Bera test statistics is 67444930.89318134 and p value is 0.0
Normality test statistics is 230720.3965134577 and p value is 0.0

il_util
Jarque-Bera test statistics is 172212.83972856315 and p value is 0.0
Normality test statistics is 26218.883604465103 and p value is 0.0

open_rv_12m
Jarque-Bera test statistics is 2641945.0804055533 and p value is 0.0
Normality test statistics is 

All variables follow the normal distribution, as such, I proceed with the analysis.

In [0]:
cont_sample_scaled.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,emp_length_int,term_int,mths_since_loan_issue,mths_since_earliest_cr,good_bad
0,-0.275501,-0.275002,-0.272950,-0.753933,-0.169730,-0.112557,0.452850,-0.354608,-0.647547,0.000000,-0.815675,-0.349276,-0.371348,-0.138904,-0.347731,-0.588249,-0.588190,0.154094,0.155923,0.387343,-0.585498,-0.124503,-0.186906,-0.179068,1.137712,-0.120698,0.0,-0.061429,-0.133941,-0.550992,-1.041750e+00,9.374643e-02,1.816751,1.153398,-6.542568e-01,0.301935,5.703958e-01,-0.243686,-3.667681e-01,0.003858,2.400560e-01,-0.493669,-1.600691e-02,0.244125,-0.022814,0.153886,-0.353365,-0.620189,1.017159,-0.078864,-0.01813,0.544789,-0.739462,-0.292568,-0.032013,-0.297903,0.317378,0.171989,-0.377617,-0.732683,-1.089031,-0.926062,-1.020911,0.761983,-0.919355,-1.137479,-1.102347,-0.827926,-0.025772,-0.05209,-0.169742,0.515969,0.660280,0.213412,-0.350283,-0.126898,-0.598969,0.070209,-0.603847,0.222224,-0.258713,-0.635836,-0.878214,-0.654022,0.366814
1,0.430918,0.431597,0.433360,-1.162576,0.543166,0.108962,1.184307,-0.354608,-0.647547,0.000000,2.021875,-0.349276,2.047801,0.390809,2.067088,1.346325,1.346608,-0.657680,-0.656199,-0.592392,-0.543406,-0.124503,-0.186906,-0.179068,-0.464107,-0.120698,0.0,-0.061429,-0.133941,-0.219812,-1.041750e+00,3.899610e+00,-0.931437,-0.457888,-2.072949e-01,0.245712,-2.985498e-01,-0.243686,-8.611072e-01,2.681972,3.007098e-01,1.917544,-1.600691e-02,-0.699995,-1.089647,-0.808398,-0.542491,1.164568,0.257171,-0.078864,-0.01813,3.011209,1.162338,-0.292568,0.078140,-0.828083,-0.497529,1.784370,-0.377617,1.891165,0.714811,1.079778,0.492709,3.499749,0.601643,0.127234,0.750374,2.043717,-0.025772,-0.05209,-0.169742,-0.598512,0.660280,0.564188,-0.350283,-0.126898,-0.048899,1.132068,3.106599,0.557096,-0.258713,-0.635836,-1.110473,0.766171,0.366814
2,0.104878,0.105474,0.107371,-1.156384,0.195171,0.271843,-0.789441,-0.354608,-0.647547,0.000000,0.071059,1.414130,-0.109572,0.629383,0.235156,-0.588249,-0.588190,0.460309,0.462269,0.808620,-0.762947,-0.124503,-0.186906,-0.179068,2.014451,-0.120698,0.0,-0.061429,-0.133941,0.637047,1.241680e-16,-1.877937e-16,0.000000,0.000000,-1.764364e-16,0.000000,8.232306e-16,0.000000,2.195307e-16,0.000000,8.619423e-16,-0.378023,1.888674e-16,0.000000,0.000000,0.153886,0.525297,-0.466409,0.608754,-0.078864,-0.01813,-0.476463,0.462225,-0.409116,-0.142166,-0.297903,0.003952,-0.007165,-0.377617,0.579241,0.414171,0.411165,1.790097,-0.743788,0.384358,1.139005,0.441588,0.069463,-0.025772,-0.05209,-0.169742,-0.598512,0.660280,0.494033,2.403678,-0.126898,0.457126,-0.479879,-0.352665,-0.626479,1.129438,-0.635836,0.701146,0.480013,0.366814
3,-0.525465,-0.525030,-0.522875,-1.177023,-0.481295,0.095931,-0.481460,-0.354608,-0.647547,2.642155,-0.993022,-0.349276,-0.596178,-1.404556,-0.347731,-0.588249,-0.588190,-0.686328,-0.684860,-0.704823,-0.639905,-0.124503,1.059660,1.116638,-0.510049,-0.120698,0.0,-0

I now check the relationship between the target variable (good_bad) and the continous variables. I drop variables that are highly inter-correlated within the feature set. In other words, I drop variables based on multicollinearity. The remaining set of variables will be used in building the credit risk model and their effects analyzed in conjuction with the categorical variables that have been excluded thus far. 

Because there are categorical variables that are yet to be analyzed alongside their continous counterparts, it is not safe to drop variables just yet based on their weak correlation with the target. Variables showing weak correlation now may be significant in predicting loan default, as such, I carry them on and analyze their full effect in the eventual model, however, I drop multicollinear variables. 

In [0]:
# Get a correlation matrix

matrix = cont_sample_scaled.corr()

In [0]:
matrix

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,emp_length_int,term_int,mths_since_loan_issue,mths_since_earliest_cr,good_bad
loan_amnt,1.000000,0.999753,0.999031,0.100890,0.945620,0.144004,0.042696,-0.007750,-0.026790,-0.010851,0.183195,-0.057652,0.320969,0.096808,0.199580,0.473329,0.473344,0.651002,0.651032,0.577787,0.583461,0.071910,0.117442,0.113817,0.318294,-0.021020,NaN,0.000350,-0.024860,0.309704,-0.019360,0.028202,0.000865,0.021660,-0.013223,0.123605,-0.065268,-0.038413,-0.030283,0.305956,0.000705,0.340273,-0.001880,0.060980,0.006946,0.001848,0.225549,0.204243,0.044798,-0.003241,0.000706,0.119480,0.162042,0.051299,0.028367,0.220307,0.031514,0.003929,-0.052382,0.188658,0.147336,0.210941,0.188769,0.081257,0.162032,0.160406,0.146817,0.179398,-0.000407,0.001113,-0.023891,-0.024207,0.088418,0.019924,-0.084714,0.010302,0.336963,0.272625,0.371006,0.204707,0.079062,0.395166,-0.062345,0.140995,-0.017731
funded_amnt,0.999753,1.000000,0.999341,0.100832,0.945965,0.143949,0.042917,-0.007670,-0.027089,-0.010804,0.183306,-0.057533,0.320804,0.096800,0.199439,0.473785,0.473800,0.650861,0.650952,0.577687,0.583250,0.071916,0.117442,0.113850,0.318290,-0.020958,NaN,0.000385,-0.024866,0.309783,-0.019365,0.028209,0.000865,0.021665,-0.013227,0.123637,-0.065285,-0.038423,-0.030291,0.306034,0.000705,0.340360,-0.001880,0.060995,0.006948,0.001826,0.225606,0.204303,0.044796,-0.003197,0.000717,0.119511,0.162083,0.051312,0.028374,0.220362,0.031527,0.003934,-0.052396,0.188706,0.147374,0.210994,0.188818,0.081278,0.162074,0.160447,0.146855,0.179442,-0.000407,0.001113,-0.023897,-0.024213,0.088440,0.019917,-0.084620,0.010376,0.337049,0.272695,0.371108,0.204760,0.079115,0.394741,-0.064122,0.140677,-0.017543
funded_amnt_inv,0.999031,0.999341,1.000000,0.100844,0.945110,0.143834,0.043390,-0.007560,-0.029035,-0.010121,0.183519,-0.057201,0.320002,0.096575,0.199349,0.474485,0.474503,0.649917,0.651171,0.576769,0.582722,0.071480,0.117029,0.113486,0.318030,-0.020780,NaN,0.000428,-0.024861,0.309609,-0.019388,0.028192,0.000845,0.021635,-0.013197,0.123559,-0.065245,-0.038437,-0.030322,0.305866,0.000690,0.340155,-0.001884,0.060958,0.006914,0.001788,0.225491,0.204226,0.044710,-0.003097,0.000724,0.119446,0.161994,0.051302,0.028367,0.220224,0.031525,0.003932,-0.052372,0.188558,0.147233,0.210840,0.188657,0.081224,0.161939,0.160304,0.146714,0.179302,-0.000437,0.001081,-0.023888,-0.024233,0.088397,0.019864,-0.084419,0.010520,0.336862,0.272514,0.370901,0.204626,0.079559,0.395111,-0.069779,0.139757,-0.016802
int_rate,0.100890,0.100832,0.100844,1.000000,0.126927,-0.035842,0.127587,0.060773,0.186047,-0.028588,-0.010841,0.053629,-0.027224,0.263729,-0.039996,0.021161,0.021186,0.099524,0.099615,-0.026609,0.413859,0.068754,0.143065,0.133926,0.077996,0.020220,NaN,0.013430,0.014730,-0.084308,0.112411,0.032505,0.127316,0.124658,-0.065978,0.031331,0.108946,0.096917,0.102920,-0.048389,0.240465,-0.187

In [0]:
# Checking for multicollinearity

for first_column in matrix.columns:
    for second_column in matrix.columns:
        if (first_column != second_column) and (matrix[first_column][second_column] <= -0.9 or matrix[first_column][second_column] >= 0.9):
            print('{} and {} are highly correlated by {}'.format(first_column, second_column, matrix[first_column][second_column]))

loan_amnt and funded_amnt are highly correlated by 0.9997532435800889
loan_amnt and funded_amnt_inv are highly correlated by 0.9990310903430786
loan_amnt and installment are highly correlated by 0.9456203923258554
funded_amnt and loan_amnt are highly correlated by 0.9997532435800889
funded_amnt and funded_amnt_inv are highly correlated by 0.9993405026619953
funded_amnt and installment are highly correlated by 0.945965377861421
funded_amnt_inv and loan_amnt are highly correlated by 0.9990310903430786
funded_amnt_inv and funded_amnt are highly correlated by 0.9993405026619953
funded_amnt_inv and installment are highly correlated by 0.9451104277553494
installment and loan_amnt are highly correlated by 0.9456203923258554
installment and funded_amnt are highly correlated by 0.945965377861421
installment and funded_amnt_inv are highly correlated by 0.9451104277553494
open_acc and num_sats are highly correlated by 0.9896859559627451
out_prncp and out_prncp_inv are highly correlated by 0.99999

The following variables are highly correlated with other variables in the feature set and I hereby drop them from analysis. And there is enough data so that dropping these records should not be of too much a concern.

In [0]:
# Make a list multicollinear variables

correlated_vars = ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'installment', 'num_sats', 'out_prncp', 'total_pymnt', 'total_pymnt_inv',
                   'total_rec_prncp', 'recoveries', 'tot_cur_bal', 'num_actv_rev_tl']

In [0]:
# Drop multicollinear variables from the analysis

selected_cont_vars = cont_sample_scaled.drop(correlated_vars, axis = 1)

In [0]:
# Quick view the resulting dataframe

selected_cont_vars.head()

,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp_inv,total_rec_int,total_rec_late_fee,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,emp_length_int,term_int,mths_since_loan_issue,mths_since_earliest_cr,good_bad
0,-0.753933,-0.112557,0.452850,-0.354608,-0.647547,0.000000,-0.815675,-0.349276,-0.371348,-0.138904,-0.347731,-0.588190,-0.585498,-0.124503,-0.179068,1.137712,-0.120698,0.0,-0.061429,-0.133941,-1.041750e+00,9.374643e-02,1.816751,1.153398,-6.542568e-01,0.301935,5.703958e-01,-0.243686,-3.667681e-01,0.003858,2.400560e-01,-0.493669,-1.600691e-02,0.244125,-0.022814,0.153886,-0.353365,-0.620189,1.017159,-0.078864,-0.01813,0.544789,-0.739462,-0.292568,-0.032013,-0.297903,0.317378,0.171989,-0.377617,-0.732683,-0.926062,-1.020911,0.761983,-0.919355,-1.137479,-1.102347,-0.025772,-0.05209,-0.169742,0.515969,0.660280,0.213412,-0.350283,-0.126898,-0.598969,0.070209,-0.603847,0.222224,-0.258713,-0.635836,-0.878214,-0.654022,0.366814
1,-1.162576,0.108962,1.184307,-0.354608,-0.647547,0.000000,2.021875,-0.349276,2.047801,0.390809,2.067088,1.346608,-0.543406,-0.124503,-0.179068,-0.464107,-0.120698,0.0,-0.061429,-0.133941,-1.041750e+00,3.899610e+00,-0.931437,-0.457888,-2.072949e-01,0.245712,-2.985498e-01,-0.243686,-8.611072e-01,2.681972,3.007098e-01,1.917544,-1.600691e-02,-0.699995,-1.089647,-0.808398,-0.542491,1.164568,0.257171,-0.078864,-0.01813,3.011209,1.162338,-0.292568,0.078140,-0.828083,-0.497529,1.784370,-0.377617,1.891165,1.079778,0.492709,3.499749,0.601643,0.127234,0.750374,-0.025772,-0.05209,-0.169742,-0.598512,0.660280,0.564188,-0.350283,-0.126898,-0.048899,1.132068,3.106599,0.557096,-0.258713,-0.635836,-1.110473,0.766171,0.366814
2,-1.156384,0.271843,-0.789441,-0.354608,-0.647547,0.000000,0.071059,1.414130,-0.109572,0.629383,0.235156,-0.588190,-0.762947,-0.124503,-0.179068,2.014451,-0.120698,0.0,-0.061429,-0.133941,1.241680e-16,-1.877937e-16,0.000000,0.000000,-1.764364e-16,0.000000,8.232306e-16,0.000000,2.195307e-16,0.000000,8.619423e-16,-0.378023,1.888674e-16,0.000000,0.000000,0.153886,0.525297,-0.466409,0.608754,-0.078864,-0.01813,-0.476463,0.462225,-0.409116,-0.142166,-0.297903,0.003952,-0.007165,-0.377617,0.579241,0.411165,1.790097,-0.743788,0.384358,1.139005,0.441588,-0.025772,-0.05209,-0.169742,-0.598512,0.660280,0.494033,2.403678,-0.126898,0.457126,-0.479879,-0.352665,-0.626479,1.129438,-0.635836,0.701146,0.480013,0.366814
3,-1.177023,0.095931,-0.481460,-0.354608,-0.647547,2.642155,-0.993022,-0.349276,-0.596178,-1.404556,-0.347731,-0.588190,-0.639905,-0.124503,1.116638,-0.510049,-0.120698,0.0,-0.061429,-0.133941,-1.041750e+00,-3.291274e-01,1.816751,0.347755,-6.045943e-01,-0.288301,5.124661e-01,-1.092359,-1.355446e+00,-0.862981,-1.238668e-01,-0.427173,-1.600691e-02,0.716185,-0.022814,-0.166876,0.736463,0.324420,-1.500746,-0.078864,-0.01813,0.063066,-1.157439,0.872921,0.078140,1.292636,0.129323,0.709449,-0.377617,-0.732683,-0.591755,-0.804680,0.077541,-1.136640,-0.884536,-1.102347,-0.025772,-0.05209,-0.169742,1.073209,0.099011,-1.189689,-0.350283,-0.126898,0.008510,-0.452306,-0.149956,-0.250089,1.129438,-0.635836,-0.042082,-0.728211,-2.726180
4,-0.229714,-0.040889,-0

The remaining variables are the selected continous variables that will be used in modeling credit risk.

In [0]:
# Make a list of selected continous variables

cont_vars_selected = []
for column_name in selected_cont_vars:
  cont_vars_selected.append(column_name)

In [0]:
# List of selected continous vars

cont_vars_selected

['int_rate',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'out_prncp_inv',
 'total_rec_int',
 'total_rec_late_fee',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'policy_code',
 'acc_now_delinq',
 'tot_coll_amt',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_inq',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'nu

This concludes analysis of continous variables from the lending club dataset.